In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

train_dir='../dataset/cifar_10_small/train'
test_dir='../dataset/cifar_10_small/test'

train_aeroplane_dir= os.path.join(train_dir,'aeroplane')
train_bird_dir=os.path.join(train_dir,'bird')
train_car_dir= os.path.join(train_dir,'car')
train_cat_dir=os.path.join(train_dir,'cat')

test_aeroplane_dir= os.path.join(test_dir,'aeroplane')
test_bird_dir=os.path.join(test_dir,'bird')
test_car_dir= os.path.join(test_dir,'car')
test_cat_dir=os.path.join(test_dir,'cat')

print('훈련용 aeroplane 이미지 전체 개수:', len(os.listdir(train_aeroplane_dir)))
print('훈련용 bird 이미지 전체 개수:', len(os.listdir(train_bird_dir)))
print('훈련용 car 이미지 전체 개수:', len(os.listdir(train_car_dir)))
print('훈련용 cat 이미지 전체 개수:', len(os.listdir(train_cat_dir)))

print('테스트용 aeroplane 이미지 전체 개수:', len(os.listdir(test_aeroplane_dir)))
print('테스트용 bird 이미지 전체 개수:', len(os.listdir(test_bird_dir)))
print('테스트용 car 이미지 전체 개수:', len(os.listdir(test_car_dir)))
print('테스트용 cat 이미지 전체 개수:', len(os.listdir(test_cat_dir)))



# parameter
batch_size=20
learning_rate=0.0001
epochs=10


1 Physical GPUs, 1 Logical GPUs
훈련용 aeroplane 이미지 전체 개수: 5000
훈련용 bird 이미지 전체 개수: 5000
훈련용 car 이미지 전체 개수: 5000
훈련용 cat 이미지 전체 개수: 5000
테스트용 aeroplane 이미지 전체 개수: 1000
테스트용 bird 이미지 전체 개수: 1000
테스트용 car 이미지 전체 개수: 1000
테스트용 cat 이미지 전체 개수: 1000


In [5]:

### data
# 모든 이미지를 1/255로 스케일을 조정합니다
agumentation_train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                    rotation_range=40,
                                    width_shift_range=0.2, ## 가로로 이동 비율
                                    height_shift_range=0.2,  ## 세로로 이동 비율
                                    shear_range=0.2,  # 전단의 강도
                                    zoom_range=0.2, ## 확대 와 축소 범위 [1-0.2 ~ 1+0.2 ]
                                    horizontal_flip=True)  ## 수평기준 플립

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)


In [6]:

train_generator = agumentation_train_datagen.flow_from_directory(
        directory=train_dir,         # 타깃 디렉터리
        target_size=(150, 150),      # 모든 이미지를 150 × 150 크기로 바꿉니다
        batch_size=batch_size,
        interpolation='bilinear',  ## resize시 interpolatrion 기법
        color_mode ='rgb',
        shuffle='True',
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='categorical') # categorical , sparse , input

validataion_generator = test_datagen.flow_from_directory(
        directory=train_dir,         # 타깃 디렉터리
        target_size=(150, 150),      # 모든 이미지를 150 × 150 크기로 바꿉니다
        batch_size=batch_size,
        interpolation='bilinear',   ## resize시 interpolatrion 기법
        color_mode ='rgb',
        shuffle='True',
        class_mode='categorical',  # categorical , sparse , input
        subset='validation')  # ImageDataGenerator 옵션에서  validation_split=0.2 만큼 validataion data 생성

print(train_generator.class_indices)
test_generator = test_datagen.flow_from_directory(
        directory=test_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        shuffle='True',
        interpolation='bilinear',  ## resize시 interpolatrion 기법
        color_mode='rgb',
        class_mode='categorical', #categorical
        subset='training') # ImageDataGenerator 옵션에서  validation_split=0.2 기준을 뺀 0.8만큼 test data 생성

for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break




Found 20000 images belonging to 4 classes.
Found 4000 images belonging to 4 classes.
{'aeroplane': 0, 'bird': 1, 'car': 2, 'cat': 3}
Found 3200 images belonging to 4 classes.
배치 데이터 크기: (20, 150, 150, 3)
배치 레이블 크기: (20, 4)


In [8]:

## 모델
input_Layer = tf.keras.layers.Input(shape=(150,150,3))
x=tf.keras.layers.Conv2D(32,(3,3),strides=1, activation='relu')(input_Layer)
x=tf.keras.layers.MaxPool2D((2,2))(x)
x=tf.keras.layers.Conv2D(64,(3,3),strides=1,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x=tf.keras.layers.Conv2D(128,(3,3),strides=1,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x=tf.keras.layers.MaxPool2D((2,2))(x)
x=tf.keras.layers.Conv2D(256,(3,3),strides=1,activation=None,padding='same')(x)
x=tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x=tf.keras.layers.Conv2D(256,(3,3),strides=1,activation='relu',padding='same')(x)
x=tf.keras.layers.MaxPool2D((2,2))(x)
x=tf.keras.layers.Flatten()(x)
x= tf.keras.layers.Dense(512, activation='relu')(x)
Out_Layer= tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()

loss_function=tf.keras.losses.categorical_crossentropy
optimize=tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
metric=tf.keras.metrics.categorical_accuracy
model.compile(loss=loss_function,
              optimizer=optimize,
              metrics=[metric])

## generator는 입력과 타싯의 배치를 끝없이 반환한다.
## 데이터가 끝없이 생성되기 때문에 모델에 하나의 에포크를 정의할때 제너레이터로부터 얼만큼 많은 샘플을 뽑을지 전달해야함
## steps_pr_epoch 가 100이면 위에서 선언된 배치 수만큼의 인풋/아웃풋데이터가 생성되어 학습이 된다.
##즉, 배치가 20이면 20의 배치인 데이터가 100번만큼 생성되어 학습한다. 즉, 20의 배치 데이터를 100번 학습완료하면 1에포크
## 단, 20의 배치데이터를 생성할때마다 랜덤적으로 생성한다.
## 일반적으로 배치사이즈/전체 데이터 길이를 steps_per_epoch로 설정한다.
with tf.device('/cpu:0'):
      history = model.fit(
            train_generator,
            steps_per_epoch=(len(os.listdir(train_aeroplane_dir)) + len(os.listdir(train_bird_dir))+len(os.listdir(train_car_dir))+len(os.listdir(train_cat_dir)))/batch_size,
            epochs=epochs,
            validation_data=validataion_generator,
            validation_freq=1)


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 70, 70, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 35, 35, 256)       2951

InternalError: stream did not block host until done; was already in an error state

In [ ]:

print(" test acc : ", model.evaluate(test_generator)[1])

model.save('cifar10_augumentation_categorical_classification.hdf5')
print(history.history.keys())


In [ ]:

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()